# Naive Model

In [1]:
'''Import Statements'''

import xarray as xr
import numpy as np
import zipfile
from datetime import datetime
import pandas as pd

In [2]:
average_sla = {}

with zipfile.ZipFile("Copernicus_ENA_Satelite Maps_Training_Data-20250111T200551Z-001.zip", "r") as zip_ref:
    # List all files in the zip
    archive = zip_ref.namelist()

    for file in archive:
        zip_ref.extract(file)
        date = file.split('_')[6]
        date_data = xr.open_dataset(file)
        sla_date = np.mean(date_data.variables["sla"][:])
        average_sla[date] = sla_date
        

In [5]:
asla_key = average_sla.keys()
asla_values = list(average_sla.values())
asla = {key:val.item() for key, val in zip(asla_key, asla_values)}
asla = dict(sorted(asla.items()))

asla_df = pd.DataFrame(asla.items(), columns=['date', 'average sla'])

asla_df['pred anomaly'] = (asla_df['average sla'] > 0.08) * 12

asla_df

,date,average sla,pred anomaly
0,19930101,-0.002769,0
1,19930102,-0.001877,0
2,19930103,-0.001213,0
3,19930104,-0.000957,0
4,19930105,-0.000709,0
...,...,...,...
7297,20131227,0.062840,0
7298,20131228,0.062716,0
7299,20131229,0.062040,0
7300,20131230,0.061426,0


In [6]:
labels = pd.read_csv("training_anomalies_merged.csv")

labels[['t', 'anomaly']].groupby("t").sum()
labels['datetime'] = [datetime.strptime(time, r"%Y-%M-%d") for time in labels['t']]
labels['date'] = [datetime.strftime(time, r"%Y%M%d") for time in labels['datetime']]

full_asla = asla_df.merge(labels[['date', 'anomaly']], how='left', on='date')

full_asla = full_asla.fillna(0)

correct = 0
guesses = 0
fp = 0
tp = 0
fn = 0
tn = 0

for index, row in full_asla.iterrows():
    if row['pred anomaly'] == 0:
        fn += row['anomaly']
        tn += 12 - row['anomaly']
    else:
        fp += 12 - row['anomaly']
        tp += row['anomaly']
    guesses += 12

print((tn + tp)/ guesses)

precision = tp / (tp + fp)
recall = tp / (tp + fn)

f1 = 2 * (precision * recall) / (precision + recall)
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

f1, tpr, fpr

0.7641224665804226


(0.05318646734854446, 0.2991150442477876, 0.22534477228992944)